## Block Matching Regisrtation

This notebook outputs transformaed pointclouds + alignmnet matrices + warped images. 
This notebut is run in WSL since it is using ants , which is not available for windows. 

For vizualisation, go to " NC_Control_View" notebook , which is run from windows ... 

In [33]:
import sys
# win:
# project_path = 'D:/Code/repos/gad1b-redistribution/src'
# wsl:
project_path = '/mnt/d/Code/repos/pwreg'
sys.path.insert(1, f'{project_path}/pwreg')

from core.core import *
from utils.utils import *
from utils.synapse import *
from visualize.viewers import *


import pickle
import pandas as pd

import inspect
# print(inspect.getsource(Something))

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
project_path

'/mnt/d/Code/repos/pwreg'

### Global image alignment 
( TODO : allow rigid ) 

In [4]:
mask_tp2 = {'zmin': 51,'zmax': 164 + 1}
mask_tp1 = {'zmin': 51,'zmax': 168 + 1} 

In [10]:
spec = {
    'fixed':
        {'filename' : f'{project_path}/data/raw/img/no_change_control/Image_1-20FJ.ome.tif',
         'resolution' : [0.658, 0.23, 0.23],
         'mask' : mask_tp1,
         'img' : None, 
         'info' : None},
    'moving':
        {'filename' : f'{project_path}/data/raw/img/no_change_control/Image_1-20FM.ome.tif',
         'resolution' : [0.658, 0.23, 0.23],
         'mask' : mask_tp2,
         'img' : None, 
         'info' : None}
        }

imp = ImagePair.from_dict(spec)
imp.shape

{'fixed': (118, 1024, 1024), 'moving': (114, 1024, 1024)}

In [11]:
# put the whole image size here
imp.register(fixed_size=[118,1024,1024], moving_size=[114,1024,1024], fixed_overlap=0, moving_overlap=0, verbose = True)


fixed : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (118, 1024, 1024)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

moving : ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (114, 1024, 1024)
	 Spacing    : (0.658, 0.23, 0.23)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]



In [21]:
imp.save(f'{project_path}/data/processed/registered/no_change_control/testing_April/test_imp/global', padding = 0, info = 'Global alignment, after cutting ')

### Block affine registration

In [23]:
# use the pre-registered 
imp2 = ImagePair(imp.fixed, imp.alignment[0].warp(interpolator = 'linear'))
imp2.shape

{'fixed': (118, 1024, 1024), 'moving': (118, 1024, 1024)}

In [25]:
# put the whole image size here
block_size = [18,102,102]
overlap = [3,7,7]
imp2.register(fixed_size=block_size, moving_size=block_size, fixed_overlap=overlap, moving_overlap=overlap, verbose = False)

Let's try with the overlap:

In [26]:
imp2.save(f'{project_path}/data/processed/registered/no_change_control/testing_April/test_imp/pw_after_glob', padding = [3,10,10], info = 'Pw registration, after global, after cutting')

### Global points registration

In [161]:
segmentation_folder = '/mnt/d/Code/repos/psd95_segmentation/models/two_small_cnns/predictions/Gad1b_2021/Sequential_2tp_control/'

ptc1 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20GA_12-22-2021_2CNN_gui.csv', prob_thr = 0.6, units='pix', resolution=[0.658, 0.23, 0.23])
ptc2 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv', prob_thr = 0.0, units='pix', resolution=[0.658, 0.23, 0.23])

numB = ptc1.num_points
numA = ptc2.num_points
print(f"Number of points before and after : {numB}, {numA}")

ptc1 = ptc1.crop(mask_tp1, units='pix')
ptc1 = ptc1.recenter([mask_tp1['zmin'],0,0], units = 'pix')

ptc2 = ptc2.crop(mask_tp2, units='pix')
ptc2 = ptc2.recenter([mask_tp2['zmin'],0,0], units = 'pix')

print(f"Number of points after cropping : {ptc1.num_points}, {ptc2.num_points}")

Number of points before and after : 10733, 427510
Number of points after cropping : 10733, 405267


In [162]:
global_transform = from_json(AffineTransform, f'{project_path}/data/processed/registered/no_change_control/testing_April/test_imp/global/transform_list.json')
ptc2 = ptc2.transform(global_transform[0])

In [166]:
sp = PwSynapsePair([ptc1], [ptc2], 1.5, numB, numA)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.48s/it]


In [169]:
max_pair = np.max(sp.match['per_idx1'][n_in_blocks], axis = 1)
n_in_blocks = np.sum(max_pair>-2)
n_lost = np.sum(max_pair==-1)
l_frac = n_lost/n_in_blocks
print(f"N in blocks {n_in_blocks}\nN lost {n_lost}\nLost fraction: {l_frac}")

N in blocks 10187
N lost 620
Lost fraction: 0.06086188279179346


In [170]:
def loss_summary(sp):
    max_pair = np.max(sp.match['per_idx1'], axis = 1)
    n_in_blocks = np.sum(max_pair>-2)
    n_lost = np.sum(max_pair==-1)
    l_frac = n_lost/n_in_blocks
    print(f"N in blocks {n_in_blocks}\nN lost {n_lost}\nLost fraction: {l_frac}")
loss_summary(sp)

N in blocks 10733
N lost 753
Lost fraction: 0.07015745830615858


In [154]:
to_json()

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:16<00:00, 16.06s/it]

N in blocks 10733
N lost 1315
Lost fraction: 0.12251933289853723


In [171]:
sp.lost

[Number of points : 753
 Resolution : [0.7 0.2 0.2]
 Coordinates :
 - pixels
 [[29.0 163.0 184.0]
  [30.0 183.0 208.0]
  [30.0 189.0 197.0]
  ...
  [111.0 998.0 768.0]
  [111.0 1010.0 806.0]
  [112.0 993.0 872.0]]
 - physical units
 [[19.1 37.5 42.3]
  [19.7 42.1 47.8]
  [19.7 43.5 45.3]
  ...
  [73.0 229.5 176.6]
  [73.0 232.3 185.4]
  [73.7 228.4 200.6]]]

In [ ]:
spec = {'filename' : f'{project_path}/data/raw/img/no_change_control/Image_1-20FJ.ome.tif',
         'resolution' : [0.658, 0.23, 0.23],
         'mask' : mask_tp1,
         'img' : None, 
         'info' : None}

fixed = Image.from_dict(spec)
print(fixed.shape)

block_size = [118,1024,1024]
overlap = 0

blocks = fixed.split(block_size,overlap)

### Block points registration

In [155]:
segmentation_folder = '/mnt/d/Code/repos/psd95_segmentation/models/two_small_cnns/predictions/Gad1b_2021/Sequential_2tp_control/'

ptc1 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20GA_12-22-2021_2CNN_gui.csv', prob_thr = 0.6, units='pix', resolution=[0.658, 0.23, 0.23])
ptc2 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv', prob_thr = 0.0, units='pix', resolution=[0.658, 0.23, 0.23])

numB = ptc1.num_points
numA = ptc2.num_points
print(f"Number of points before and after : {numB}, {numA}")

ptc1 = ptc1.crop(mask_tp1, units='pix')
ptc1 = ptc1.recenter([mask_tp1['zmin'],0,0], units = 'pix')

ptc2 = ptc2.crop(mask_tp2, units='pix')
ptc2 = ptc2.recenter([mask_tp2['zmin'],0,0], units = 'pix')

print(f"Number of points after cropping : {ptc1.num_points}, {ptc2.num_points}")

Number of points before and after : 10733, 427510
Number of points after cropping : 10733, 405267


In [156]:
spec = {'filename' : f'{project_path}/data/raw/img/no_change_control/Image_1-20FJ.ome.tif',
         'resolution' : [0.658, 0.23, 0.23],
         'mask' : mask_tp1,
         'img' : None, 
         'info' : None}

fixed = Image.from_dict(spec)
print(fixed.shape)

block_size = [18,102,102]
overlap = [3,7,7]

blocks = fixed.split(block_size,overlap)

(118, 1024, 1024)


/mnt/d/Code/repos/pwreg/pwreg/core/core.py:112: UserWarning:

Specified voxel size + overlap don't cover the whole image.Image size is [ 118 1024 1024], block size [ 18 102 102], overlap [3 7 7] results in [7.7 10.7 10.7] blocks.
Leaving some image out. 



In [157]:
global_transform = from_json(AffineTransform, f'{project_path}/data/processed/registered/no_change_control/testing_April/test_imp/global/transform_list.json')
pw_transforms = from_json(AffineTransform, f'{project_path}/data/processed/registered/no_change_control/testing_April/test_imp/pw_after_glob/transform_list.json')
ptc2 = ptc2.transform(global_transform[0])
ptc2 = ptc2.pw_transform(pw_transforms)

In [158]:
ptc1_in_blocks = ptc1.split(blocks)
ptc2_in_blocks = [ptc.fit_block(blc) for ptc, blc in zip(ptc2, blocks)]

In [148]:
sp = PwSynapsePair(ptc1_in_blocks, ptc2_in_blocks, 1.5, numB, numA)

100%|█████████████████████████████████████████████████████████████████████████████████| 700/700 [00:09<00:00, 76.12it/s]


In [149]:
def loss_summary(sp):
    max_pair = np.max(sp.match['per_idx1'], axis = 1)
    n_in_blocks = np.sum(max_pair>-2)
    n_lost = np.sum(max_pair==-1)
    l_frac = n_lost/n_in_blocks
    print(f"N in blocks {n_in_blocks}\nN lost {n_lost}\nLost fraction: {l_frac}")
loss_summary(sp)

N in blocks 10187
N lost 615
Lost fraction: 0.06037106115637577


In [129]:
def show_repeats(match_matrix, rep):
    pairs = np.max(match_matrix, axis = 1)
    only_syns = pairs[pairs >-1]
    in_syns, c = np.unique(only_syns, return_counts=True)
    overused = in_syns[c == rep]
    print(f" idx : pairs")
    for idx in overused:
        locations = np.where(pairs == idx)[0]
        for loc in locations: 
            all_line = match_matrix[loc,:]
            only_meaning = all_line[all_line>-2]
            print(f"{loc}: {only_meaning}")
            
show_repeats(sp.match['per_idx1'], 2)

 idx : pairs
148: [151  -1]
150: [151]
1306: [1360   -1]
1322: [1360]
1421: [  -1 1499]
1439: [1499   -1]
2924: [  -1 3202]
7117: [3202]
3617: [4046   -1]
3631: [4046]
3836: [4290   -1]
3855: [4290]
5042: [5573 5573]
5060: [  -1   -1 5573 5573]
6347: [7042   -1]
6385: [7042]
7493: [  -1 8309]
7523: [8309 8309]
7924: [8803 8783]
7932: [8803 8803]
8040: [8955   -1]
8091: [8955]
8042: [8958]
8092: [8958]
8958: [9881]
8964: [  -1 9881]
8970: [9912 9911]
9000: [9912]
9520: [10515]
9530: [   -1 10515]
10038: [11116]
10052: [11116]
10452: [11545    -1]
10461: [11545]


In [159]:
sp = PwSynapsePair(ptc1_in_blocks, ptc2_in_blocks, 1.0, numB, numA)
loss_summary(sp)

100%|█████████████████████████████████████████████████████████████████████████████████| 700/700 [00:08<00:00, 82.41it/s]


N in blocks 10187
N lost 1090
Lost fraction: 0.10699911652105625


In [160]:
n_in_blocks = np.max(sp.match['per_idx1'], axis = 1)>-2
n_in_blocks.shape

(10733,)

In [112]:
orig_du = pairs[in_syns[c ==2]]

array([False, False, False, ..., False, False, False])

In [113]:
sp.match['per_idx1'][double_used,:]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 10733 but corresponding boolean dimension is 7815

In [ ]:
PointsBlockView(ptc2_in_blocks, voxels_f, 10).to_json(f'{project_path}/data/processed/registered/no_change_control/ptc2_block_ZYX.json')

In [ ]:
ptc1_global = Points.from_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_global_tansformed_to_ptc2_TS_resolution_ZYX.json')
ptc1_to_ptc2_in_blocks = ptc1_global.pw_transform(pairs)
PointsBlockView(ptc1_to_ptc2_in_blocks, voxels_f, 10).to_json(f'{project_path}/data/processed/registered/no_change_control/ptc1_to_ptc2_block_ZYX.json')

In [ ]:
a = []

In [ ]:
a

[]

In [25]:
a[2] = 9

IndexError: list assignment index out of range

In [67]:
ptc1_test = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20GA_12-22-2021_2CNN_gui.csv', prob_thr = 0.6, units='pix', resolution=[0.658, 0.23, 0.23])
ptc2_test = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv', prob_thr = 0.6, units='pix', resolution=[0.658, 0.23, 0.23])
ptc1_test.num_points

10733

In [68]:
ptc2_test.num_points

11848

In [172]:
spec = {'filename' : f'{project_path}/data/raw/img/no_change_control/Image_1-20FM.ome.tif',
         'resolution' : [0.658, 0.23, 0.23],
         'mask' : mask_tp2,
         'img' : None, 
         'info' : None}

imp = Image.from_dict(spec)
imp.shape

(114, 1024, 1024)

In [188]:

impb = imp.blur([0.5,1,1])
impb.imwrite(f'{project_path}/data/processed/registered/no_change_control/testing_April/test_blur_05_1_1.tif')

In [187]:
imdog = imp.dog([0.5,1,1],[1,2,2])
imdog.imwrite(f'{project_path}/data/processed/registered/no_change_control/testing_April/test_dog.tif')

In [176]:
ptc2_full = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20G6_12-22-2021_2CNN_gui.csv', prob_thr = 0.0, units='pix', resolution=[0.658, 0.23, 0.23])
ptc2_full = ptc2_full.crop(mask_tp2, units='pix')
ptc2_full = ptc2_full.recenter([mask_tp2['zmin'],0,0], units = 'pix')

to_json(ptc2_full, f'{project_path}/data/processed/registered/no_change_control/testing_April/all_candidates_tp2.json')

In [180]:
from scipy import ndimage as ndi


def local_maxima_3D(data, order=1):
    """Detects local maxima in a 3D array

    Parameters
    ---------
    data : 3d ndarray
    order : int
        How many points on each side to use for the comparison

    Returns
    -------
    coords : ndarray
        coordinates of the local maxima
    values : ndarray
        values of the local maxima
    """
    size = 1 + 2 * order
    footprint = np.ones((size, size, size))
    footprint[order, order, order] = 0

    filtered = ndi.maximum_filter(data, footprint=footprint)
    mask_local_maxima = data > filtered
    coords = np.asarray(np.where(mask_local_maxima)).T
    # values = data[mask_local_maxima]

    return coords

In [190]:
coords = local_maxima_3D(impb.img, order=2)
coords.shape

(379359, 3)

In [182]:
ptc2_lm = Points(coords, units='pix', resolution=[0.658, 0.23, 0.23])
to_json(ptc2_lm, f'{project_path}/data/processed/registered/no_change_control/testing_April/local_maximum_small_blur_tp2.json')

In [192]:
ptc1 = Points.from_predictions(f'{segmentation_folder}ML_segmentation_1-20GA_12-22-2021_2CNN_gui.csv', prob_thr = 0.6, units='pix', resolution=[0.658, 0.23, 0.23])
ptc1 = ptc1.crop(mask_tp1, units='pix')
ptc1 = ptc1.recenter([mask_tp1['zmin'],0,0], units = 'pix')

global_transform = from_json(AffineTransform, f'{project_path}/data/processed/registered/no_change_control/testing_April/test_imp/global/transform_list.json')
ptc2_lm = ptc2_lm.transform(global_transform[0])

sp = PwSynapsePair([ptc1], [ptc2_lm], 1.5, numB, numA)
loss_summary(sp)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:13<00:00, 13.27s/it]

N in blocks 10733
N lost 1022
Lost fraction: 0.09522034845802664


In [185]:
coords = local_maxima_3D(impb.img, order=2)
coords.shape

(71567, 3)

In [191]:
ptc2_lm = Points(coords, units='pix', resolution=[0.658, 0.23, 0.23])
to_json(ptc2_lm, f'{project_path}/data/processed/registered/no_change_control/testing_April/local_maximum_small_blur_order2_tp2.json')